# 1) Determining forest gain and loss for subareas between the two closest dates

    This code processes Sentinel-2 satellite imagery to detect changes in forest cover, calculating areas of forest loss and gain, and generates change maps for areas of interest with a high classification accuracy.

In [ ]:
import rasterio
import numpy as np
import os
import re
from datetime import datetime

input_folder = '/path/to/tif_files/sentinel2'
tiles_accuracy_file = '/path/to/tiles_accuracy.txt'
output_folder = '/path/to/save/gain_loss"
summary_file_path = os.path.join(output_folder, "gain_loss.txt")

def read_raster(file_path):
    with rasterio.open(file_path) as dataset:
        data = dataset.read(1)
    return data

def compare_forest_change(base_map, new_map):
    loss = (base_map == 6) & (new_map != 6)
    gain = (base_map != 6) & (new_map == 6)
    return loss, gain

def calculate_area(pixels, pixel_size=400):  # 20m x 20m = 400m^2 per pixel
    area = np.sum(pixels) * pixel_size
    return area

def create_change_map(base_path, new_path, output_path):
    base_map = read_raster(base_path)
    new_map = read_raster(new_path)

    loss, gain = compare_forest_change(base_map, new_map)

    change_map = np.zeros(base_map.shape, dtype=np.int8)
    change_map[loss] = 1  # Utrata
    change_map[gain] = 2  # Przyrost

    loss_area = calculate_area(loss)
    gain_area = calculate_area(gain)

    with rasterio.open(base_path) as src:
        profile = src.profile
        profile.update(dtype=rasterio.int8, count=1, nodata=None)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(change_map, 1)

    print(f"Change map written to {output_path}")
    return loss_area, gain_area

def find_aoi_files(folder_path):
    aoi_files = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.tif'):
                match = re.search(r'AOI0_(\d+)_\d{4}-\d{2}-\d{2}', file)
                if match:
                    aoi = match.group(1)
                    if aoi not in aoi_files:
                        aoi_files[aoi] = []
                    aoi_files[aoi].append(os.path.join(root, file))
    return aoi_files

def read_accuracy_file(accuracy_file_path):
    with open(accuracy_file_path, 'r') as file:
        content = file.read()

    pattern = r'Result for (.*?):.*?Class 6: ([0-9.]+)'
    matches = re.findall(pattern, content, re.DOTALL)

    valid_files = {}
    for match in matches:
        filename, f1_score = match
        if float(f1_score) >= 0.8:
            aoi_key = re.search(r'AOI0_(\d+)_\d{4}-\d{2}-\d{2}', filename).group(1)
            valid_files[aoi_key] = filename
    
    return valid_files

def process_aoi_files(aoi_files, valid_files, output_folder, summary_file_path):
    with open(summary_file_path, 'w') as summary_file:
        for aoi, files in aoi_files.items():
            if aoi not in valid_files:
                continue

            files_sorted = sorted(files, key=lambda x: datetime.strptime(x.split('_')[-1].split('.')[0], '%Y-%m-%d'))
            for i in range(len(files_sorted) - 1):
                base_file = files_sorted[i]
                new_file = files_sorted[i + 1]
                base_date = base_file.split('_')[-1].split('.')[0]
                new_date = new_file.split('_')[-1].split('.')[0]

                with rasterio.open(base_file) as base_src, rasterio.open(new_file) as new_src:
                    if base_src.shape != new_src.shape:
                        continue

                output_path = os.path.join(output_folder, f"AOI0_{aoi}_{base_date}_to_{new_date}_change_map.tif")
                loss_area, gain_area = create_change_map(base_file, new_file, output_path)

                summary_file.write('_' * 70 + '\n')
                summary_file.write(f"AOI: AOI0_{aoi}, Dates: {base_date} to {new_date}\n")
                summary_file.write(f"Sum of loss area: {loss_area} m^2\n")
                summary_file.write(f"Sum of gain area: {gain_area} m^2\n")


valid_files = read_accuracy_file(tiles_accuracy_file)
aoi_files = find_aoi_files(input_folder)
process_aoi_files(aoi_files, valid_files, output_folder, summary_file_path)
print("Completed.")
